# MFE 413: Data Analytics and Machine Learning

## Problem Set 5

### Cohort 2 Group 5
* Chen Yechao
* Chopra Aryan
* Gonzalez Rivadeneira Sara
* Jensen Mathieu
* Kumar Pranay

In [3]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.linear_model import LinearRegression, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

# Question 1:

### Simulate 1500 realizations of two uncorrelated standard Normal variables. Call the simulated variables $x_1$ and $x_2$ and use these simulated variables as your predictors for y. Simulate 1500 outcomes for y for each of the two models:

### a) $1.5 x_1 - 2x_2 + \epsilon$

### b) $y=1.5x_1 -2x_2 + \epsilon$, if $x_1 < 0$
### $y= 1.5 $ ln $x_1 + \epsilon$, if $x_1 \geq 0$

In [ ]:
n_samples = 1500
n_train = 1000
n_test = 500
n_repeats = 500
rng = np.random.default_rng(seed=42)
mse_a = {'OLS': [], 'RF': [], 'XGB': []}
mse_b = {'OLS': [], 'RF': [], 'XGB': []}

# Question 2:

### Our goal in this exercise is to predict house prices in Boston (medv) given 11 explanatory variables (columns 1 through 11). Use the first 400 observations as your training sample and observations 401-506 as your test sample. 

#### (a) Use random forest with n_estimators=250 and max_depth=10. Once you run the random forest, use Python’s rf.predict function to obtain predicted values for the test sample. What is the MSE of the prediction? Compare this to the benchmark MSE generated by a model that has as its predicted house value the mean house value in the test sample. As in the class notes, also report the Pseudo-R2 implied by these MSEs. 

In [8]:
data = pd.read_csv('housing.csv')

training = data.iloc[0:399]
predictingTraining = training.drop(columns = ['medv'])
labelsTraining = training.medv

testing = data[400:]
predictingTesting = testing.drop(columns = ['medv'])
labelsTesting = testing.medv

In [11]:
rf = RandomForestRegressor(n_estimators=250,
                           max_depth=10)
rf.fit(predictingTraining, labelsTraining)
prediction = rf.predict(predictingTesting)

In [22]:
MSE_RF = np.mean(np.square((prediction - labelsTesting)))

MSE_Baseline = np.mean(np.square((labelsTesting - np.mean(labelsTraining))))
lm = LinearRegression().fit(predictingTraining, labelsTraining)
lm_prediction = lm.predict(predictingTesting)
MSE_lm = np.mean(np.square((lm_prediction - labelsTesting)))
PseudoR2 = 1 - MSE_lm / MSE_Baseline

print('MSE of this prediciton is', MSE_RF)
print('The psuedo R^2 is', PseudoR2)

MSE of this prediciton is 16.9130881228398
The psuedo R^2 is 0.6323003759967234


#### (b) Repeat the same exercise as above using XGBoost with learning_rate=0.1, gamma=0, max_depth=6. Use 10 folds and 200 rounds for the cross-validation procedure. Make sure that the output of the cross-validation procedure does not appear in your final write-up.

In [37]:
xgb_regressor = xgb.XGBRegressor(booster = "gbtree",
                                 objective = "reg:squarederror",
                                 n_estimators = 200,
                                 reg_lambda = 10,
                                 gamma = 0,
                                 learning_rate = 0.1,
                                 max_depth=6)
xgb_parm = xgb_regressor.get_xgb_params()

In [40]:
xgb_train = xgb.DMatrix(predictingTraining, label = labelsTraining)

xgb_cvresult = xgb.cv(xgb_parm,
                      xgb_train,
                      metrics = "rmse",
                      num_boost_round=200,
                      nfold = 10,
                      early_stopping_rounds=25)

xgb_regressor.set_params(n_estimators = xgb_cvresult.shape[0])
xgb_regressor.fit(predictingTraining, labelsTraining)

xgb_prediction = xgb_regressor.predict(predictingTesting)

MSE_xgb = np.mean(np.square((xgb_prediction - labelsTesting)))
PseudoR2_xgb = 1 - MSE_xgb / MSE_Baseline

print('MSE of this prediciton is', MSE_xgb)
print('The psuedo R^2 is', PseudoR2_xgb)

MSE of this prediciton is 20.36334406235749
The psuedo R^2 is 0.8023093481951917


#### (c)  Repeat the exercise in part (a) using elastic net regression (l1_ratio=0.5). Use a cross-validation procedure to find an optimal lambda (alpha). For that exercise, split the training sample into quarters (i.e., the 4-fold cross-validation). Comment on the performance of the linear model relative to decision trees. In particular, get the MSE for the test sample and compute the Pseudo-R2 relative to the benchmark MSE from a). 

In [43]:
elasticNetRegression = ElasticNetCV(l1_ratio=0.5,
                                    cv=4)

elasticNetRegression.fit(predictingTraining, labelsTraining)
elasticPrediction = elasticNetRegression.predict(predictingTesting)

MSE_elastic = np.mean(np.square((elasticPrediction - labelsTesting)))
PseudoR2_elastic = 1 - MSE_elastic / MSE_Baseline

print('MSE of this prediciton is', MSE_elastic)
print('The psuedo R^2 is', PseudoR2_elastic)

MSE of this prediciton is 27.218328546143553
The psuedo R^2 is 0.7357600453615478


#### (d)  Try to figure out what the main sources of the discrepancy between Elastic Net and the decision trees are. That is, what is the non-linearity? 